In [1]:
import cgt
from cgt import nn, utils, display
import numpy as np, numpy.random as nr
from numpy.linalg import norm
from param_collection import ParamCollection



/Users/zoe/cgt/cgt/__init__.py:15: UserWarning: 
    Could not import the compiled extension module cycgt
    Only pure python mode is available. If you have compiled "
    this extension (via 'make'), you may need to add build/lib 
    to your PYTHONPATH. Try 'import cycgt' to debug the problem further.
  warnings.warn(msg, UserWarning)


In [2]:
k_in = 2
k_h = 2
size_x = 3
size_mem = 4
size_batch = 4

In [12]:
x = cgt.matrix(fixed_shape=(size_batch, size_x))
xform_h_param = nn.TensorParam((2 * k_h, size_mem), name="rotxform")
xform_h_non = xform_h_param.weight
xform_h_norm = cgt.norm(xform_h_non, axis=1, keepdims=True)
xform_h = cgt.broadcast('/', xform_h_non, xform_h_norm, "xx,x1")

prev_h = cgt.matrix(fixed_shape=(size_batch, size_mem))
r_vec = nn.Affine(size_x, 2 * k_in * size_mem)(x)
r_non = cgt.reshape(r_vec, (size_batch, 2 * k_in, size_mem))
r_norm = cgt.norm(r_non, axis=2, keepdims=True)
r = cgt.broadcast('/', r_non, r_norm, "xxx,xx1")
prev_h_3 = cgt.reshape(prev_h, (size_batch, size_mem, 1))
inters_in = [prev_h_3]

colon = slice(None, None, None)

for i in xrange(2 * k_in):
    inter_in = inters_in[-1]
    r_cur = cgt.subtensor(r, [colon, i, colon])
    r_cur_3_transpose = cgt.reshape(r_cur, (size_batch, 1, size_mem))
    r_cur_3 = cgt.reshape(r_cur, (size_batch, size_mem, 1))
    ref_cur = cgt.batched_matmul(r_cur_3, cgt.batched_matmul(r_cur_3_transpose, inter_in))
    inter_out = inter_in - 2 * ref_cur
    inters_in.append(inter_out)

h_in_rot = cgt.reshape(inters_in[-1], (size_batch, size_mem))
inters_h = [h_in_rot]

for i in xrange(2 * k_h):
    inter_in = inters_h[-1]
    r_cur = cgt.subtensor(xform_h, [i, colon])
    r_cur_2_transpose = cgt.reshape(r_cur, (size_mem, 1))
    r_cur_2 = cgt.reshape(r_cur, (1, size_mem))
    ref_cur = cgt.dot(cgt.dot(inter_in, r_cur_2_transpose), r_cur_2)
    inter_out = inter_in - 2 * ref_cur
    inters_h.append(inter_out)
h = inters_h[-1]
    

#h = h_in_rot

    
r_nn = nn.Module([x], [h])

In [13]:
params = r_nn.get_parameters()
pc = ParamCollection(params)
pc.set_value_flat(nr.uniform(-.1, .1, size=(pc.get_total_size(),)))
func = cgt.function([x, prev_h], h)

In [14]:
x_in = nr.uniform(-.1, .1, size=(size_batch * size_x)).reshape(size_batch, size_x)
h_in = np.zeros((size_batch, size_mem))
h_in[:, 0] = np.ones(size_batch)
h = func(x_in, h_in)
print h
print norm(h, axis=1)
print norm(h_in, axis=1)


[[ 0.46639985 -0.13617726 -0.83365595  0.26257211]
 [ 0.52262288 -0.0058302  -0.79804057  0.29993784]
 [ 0.4850145   0.03708118 -0.80431843  0.34125924]
 [ 0.63327622  0.1306653  -0.71277249  0.27174002]]
[ 0.9999997  1.         1.         0.9999997]
[ 1.  1.  1.  1.]


In [36]:
slice?